In [ ]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw
from bw2data import parameters
from sympy.physics.units import years
from sympy.stats.crv_types import LogNormalDistribution
from sympy import init_printing
import lca_algebraic as agb
from dotenv import load_dotenv

# Custom utils defined for the parameterization
from lca_algebraic import *
from lca_algebraic.stats import *

# Pretty print for Sympy
init_printing()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, lognorm, expon, beta, uniform, triang, kstest

In [ ]:
BW_PROJECT = 'lca-alg-2' # insert your project name here
bd.projects.set_current(BW_PROJECT)

EI_DB = 'ecoinvent-3.9.1-cutoff' # name of ecoinvent database in your project
# We use a separate DB for defining our foreground model / activities
USER_DB = "MyForeground"

In [ ]:
# This is better to cleanup the whole foreground model each time, and redefine it in the notebook (or a python file)
# instead of relying on a state or previous run.
# Any persistent state is prone to errors.
agb.resetDb(USER_DB)

# Parameters are stored at project level : 
# Reset them also
# You may remove this line if you import a project and parameters from an external source (see loadParam(..))
agb.resetParams()

# Overview of the databases
agb.list_databases()

## Define input parameters

lca_algebraic supports 7 distribution types: 
- Fixed: for excluding parameters from the statistical study
- Uniform: uniform distribution within the range of definition
- Triangle: null probability at the boundaries and highest probability for a default value, to be defined within this range. This type of distribution is useful for parameters for which literature only provides a range of extreme values and a usual one.
- Normal: normal distribution, capped to minimum and maximum values
- Log-normal: log-normal distribution, capped to minimum and maximum values
- Beta: beta distribution, capped to minimum and maximum values
- Statistic weight: for discrete parameters: boolean and enum

lca_algebraic supports 3 types of parameters that can be used in the inventory: 
- Float
- Boolean (e.g. discrete parameters, 0 or 1)
- Enumerated: discrete parameters set to one value among a predefined list. They are modeled internally as a linear combination of a set of exclusive boolean parameters

In [ ]:
# Example of 'float' parameters
a = agb.newFloatParam(
    'a', 
    default=0.5, min=0.2, max=2,  
    distrib=agb.DistributionType.TRIANGLE, # Distribution type, linear by default
    description="hello world",
    label="extended label for a")

b = agb.newFloatParam(
    'b',
    default=0.5, # Fixed if no min /max provided
    distrib=agb.DistributionType.FIXED,
    description="foo bar")

share_recycled_aluminium = agb.newFloatParam(
    'share_recycled_aluminium',  
    default=0.6, 
    min=0, max=1, std=0.2, 
    distrib=agb.DistributionType.NORMAL, # Normal distrib, with std dev
    description="Share of reycled aluminium")

c = agb.newFloatParam(
    'c',  
    default=0.6, std=0.2, 
    distrib=agb.DistributionType.LOGNORMAL)

beta = agb.newFloatParam(
    'beta',  
    default=0.6, std=0.2, a=2, b=5, 
    distrib=agb.DistributionType.BETA)

In [ ]:
# Enum parameters
# Example 'enum' parameter, acting like a switch between several possibilities
# Enum parameters are not Symbol themselves
# They are a facility to represent many boolean parameters at once '<paramName>_<enumValue>' 
# and should be used with the 'newSwitchAct' method 
elec_switch_param = agb.newEnumParam(
    'elec_switch_param', 
    values=["us", "eu"], # If provided as list, all possibilities have te same probability
    default="us", 
    description="Switch on electricty mix")

# Another example enum param
techno_param = agb.newEnumParam(
    'techno_param', 
    values={
        "technoA":0.4, 
        "technoB":0.1,
        "technoC":0.5}, # You can provide a statistical weight for each value, by using a dict
    default="technoA", 
    description="Choice of technology")

In [ ]:
# You can define boolean parameters, taking only discrete values 0 or 1
bool_param = agb.newBoolParam(
    'bool_param', 
    default=1)

By default, new parameters are kept in memory but also persisted in the project (unless save=False).

You can persist parameters afterwards with `persistParams()`.

You can load also load parameters from an existing database with `loadParams()`.

The persistance of parameters and the distribution is compatible with **Brightway2** and **Activity Browser**  [see documentation of stat_arrays](https://stats-arrays.readthedocs.io/en/latest/)

In [ ]:
# Load parameters previously  persisted in the dabatase.
agb.loadParams()

lca_algebraic supports several foreground / background datasets. Background datasets are considered static / non parametrized by the library : they use standard LCA method of **Brightway2**. 

Foreground databases are considered parametric and their activities are developped as functions of parameters and background activities.

In [ ]:
agb.setForeground(USER_DB)
agb.list_databases()

In [ ]:
# Save database and parameters as Bzipped JSON
#agb.export_db(USER_DB, "data/db.bw2")

In [ ]:
# Reimport DB
#agb.import_db("data/db.bw2")

A foreground database can be "frozen" to be used as a background database for a specific scenario : the parametrized amounts in the exhanges are computed for a given configuration of the parameters, and replaced by their value. The formulas are still stored in the database and not lost : the database can still be used as a foreground database until its status is changed with `setBackground(...)`.

In [ ]:
agb.freezeParams(
    USER_DB, # Name of database to freeze
    
    a=1, b=2) # custom parameter values

# Link to background activities and creation of new activities

We provide two functions for easy and fast (indexed) search of activities in reference databases : 
* **findBioAct** : Search activity in **biosphere3** db
* **findTechAct** : Search activity in **ecoinvent** db

Those methods are **faster** and **safer** than using traditionnal "list-comprehension" search : 
They will **fail with an error** if **more than one activity** matches, preventing the model to be based on a random selection of one activity.

The model is defined as a nested combination of background activities with amounts. Amounts are defined either as constant float values or algebric formulas implying the parameters defined above.

## Existing activities

In [ ]:
# Biosphere activities
ground_occupuation = agb.findBioAct('Occupation, industrial area') # Search by name
heat = agb.findBioAct('Heat, waste', categories=['air']) # Add category selector

# Technosphere activities

# You can add an optionnal location selector
alu = agb.findTechAct("aluminium alloy production, AlMg3", loc="RER")
alu_scrap = agb.findTechAct('aluminium scrap, new, Recycled Content cut-off')

# Elec 
eu_elec = agb.findTechAct("market group for electricity, medium voltage", 'ENTSO-E')
us_elec = agb.findTechAct("market group for electricity, medium voltage", 'US')

chromium = agb.findTechAct("market for chromium oxide, flakes")

## New activities

In [ ]:
# Create a new activity
activity1 = agb.newActivity(USER_DB, # We define foreground activities in our own DB
    "first foreground activity", # Name of the activity
    "kg", # Unit
    exchanges= { # We define exhanges as a dictionarry of 'activity : amount'
        ground_occupuation:3 * b, # Amount can be a fixed value 
        heat: b + 0.2  # Amount can be a Sympy expression (any arithmetic expression of Parameters)
    })

# You can create a virtual "switch" activity combining several activities with an Enum parameter
elec_mix = agb.newSwitchAct(USER_DB, 
    "elect mix", # Name
    elec_switch_param, # Sith parameter
    { # Dictionnary of enum values / activities
        "us" : us_elec, # By default associated amount is 1
        "eu" : (eu_elec, 0.8)  # You can also provide custom amout or formula with a tuple 
    })

## Copy activities

In [ ]:
# Copy and update existing activity
alu2 = agb.copyActivity(
    USER_DB, # The copy of a background activity is done in our own DB, so that we can safely update it                
    alu, # Initial activity : won't be altered
    "Aluminium 2") # New name

# Update exchanges by their name 
alu2.updateExchanges({
    
    # Update amount : the special symbol *old_amount* references the previous amount of this exchange
    "aluminium, cast alloy": agb.old_amount * (1 - share_recycled_aluminium),
    
    # Update input activity. Note also that you can use '*' wildcard in exchange name
    "electricity*": elec_mix,
    
    # Update both input activity and amount. 
    # Note that you can use '#' for specifying the location of exchange (useful for duplicate exchange names)
    "chromium#GLO" : dict(amount=4.0, input=chromium)
}) 

# Add exchanges 
alu2.addExchanges({alu_scrap :  12})

## Final inventory

In [ ]:
# we define our final model 
total_inventory = agb.newActivity(USER_DB, "total_inventory", "kg", {
    activity1 : b * 5 + a + 1, # Reference the activity we just created
    alu2: 3 * share_recycled_aluminium, 
    alu:0.4 * a})

In [ ]:
#Alternatively, you may not define the model again, but load it from the USER DB.
activity1 = agb.findActivity("first foreground activity", db_name=USER_DB)
total_inventory = agb.findActivity("total_inventory", db_name=USER_DB)
alu2 = agb.findActivity("Aluminium 2", db_name=USER_DB)

In [ ]:
agb.printAct(activity1) 

In [ ]:
agb.printAct(total_inventory)


In [ ]:
# You can also compute amounts by replacing parameters with a float value 
agb.printAct(activity1, b=1.5)

In [ ]:
# You can print several activities at once to compare them
agb.printAct(alu, alu2)

# Impact calculation

In [ ]:
# List of impacts to consider
impacts = agb.findMethods("climate change", mainCat="EF v3.0")
impacts

In [ ]:
# Definition of FU, which can be parametrized
functional_value = a + 5

In [ ]:
agb.list_databases()


## Simple first

In [ ]:
agb.compute_impacts(
    
    # Root activity of our inventory
    total_inventory, 
    
    # list of impacts to consider
    impacts, 
    
    # The impacts will be divided by the functional unit
    functional_unit=functional_value,
    
    # Parameters of the model
    a=1.0,
    elec_switch_param="us",
    share_recycled_aluminium=0.4)

In [ ]:
# You can compute several LCAs at a time and compare them:
agb.compute_impacts(
    [alu, alu2], # The models
    
    impacts, # Impacts
    
    # Parameters of the model
    share_recycled_aluminium=0.3,
    elec_switch_param="us")

## Fast computation for millions of separate samples

In [ ]:
# Fast computation for millions of separate samples
agb.compute_impacts(
    total_inventory, # The model 
    impacts, # Impacts
    functional_unit = functional_value,
    
    # Parameters of the model
    a=list(range(1, 100000)), # All lists should have the same size
    share_recycled_aluminium=1, # Those parameters are fixed
    elec_switch_param="eu")

## Split impacts

It is possible to **tag** activities and then ventilate the impacts according to the value of this "tag".
This is useful to split impact by *phase* or *sub module*.

In [ ]:
# Tag activities with a custom attribute : 'phase' in this case
alu2.updateMeta(phase= "phase a")
activity1.updateMeta(phase= "phase b")

In [ ]:
# Provide the name of the custom attribute as 'axis', the impacts are split between those
agb.compute_impacts(
    total_inventory, # The model
    impacts, # Impacts
    
    functional_unit = functional_value,
    axis="phase",

    
    # Parameters
    a=1.0,
    elec_switch_param="us",
    share_recycled_aluminium=0.4)

# Sensitivity analysis



## OAT

 Shows a **matrix of impacts x parameters** colored according to the variation of the impact in the bounds of the parameter.

In [ ]:
agb.oat_matrix(
    total_inventory, 
    impacts, 
    functional_unit=functional_value)

This functions draws a dashboard showing :
* A dropdown list, for choosing a parameter
* Several graphs of evolution of impacts for this parameter
* Full table of data
* A graph of "bars" representing the variation of each impact for this parameter (similar to the information given in oat_matrix) 

In [ ]:
agb.oat_dashboard(
    total_inventory, 
    impacts, 
    functional_unit=functional_value,
    
    # Optionnal layout parameters
    figspace=(0.5,0.5),
    figsize=(15, 15),
    sharex=True)

This method shows some limitations though: it does not provide a quantified assessment of the variance of the impacts, and it may hide the importance of some parameters, only revealed when combined with different setups of the other parameters.

## GSA with Monte-Carlo methods and Sobol indices

Similar to OAT matrix, we compute Sobol indices. they represent the ratio between the variance due to a given parameter and the total variance.

for easier comparison, we translate those relative sobol indices into "deviation / mean" importance :

$$RelativeDeviation = \frac{\sqrt{sobol(param) \times totalVariance(impact))}}{mean(impact)}$$


In [ ]:
# Show sobol indices 
agb.incer_stochastic_matrix(
    total_inventory, 
    impacts, 
    functional_unit=functional_value)

We provide a dashboard showing **violin graphs** : the exact probabilistic distribution for each impact. Together with medians of the impacts.

We provide a dashboard showing **violin graphs** : the exact probabilistic distribution for each impact. Together with medians of the impacts.

In [ ]:
agb.incer_stochastic_violin(
    total_inventory, impacts,
    functional_unit=functional_value,
    
    # Optionnal layout parameters
    figspace=(0.5,0.5),
    figsize=(15, 15),
    sharex=True, 
    nb_cols=3)

In [ ]:
# Alternatively, graphs can be shown horizontally, together with a box of statistical outcomes
agb.distrib(
    total_inventory, impacts,
    functional_unit=functional_value,
    
    # Optionnal layout parameters
    height=7, width=15,
    nb_cols=2,
    percentiles=[5, 95])

In [ ]:
# Full dashboard, including total variation of impacts 
agb.incer_stochastic_dashboard(
    model=total_inventory, 
    methods=impacts,
    functional_unit=functional_value)

# Simplified model


In [ ]:
# First, let's look at the full expression defining our model
expr, _ = agb.actToExpression(total_inventory)
expr

We provide some method to automatically select a subset of parameters, based on the **sobol indices**, and then compute simplified models for it.

We also round numerical expression to 3 digits, and we remove terms in sums that are less than 1% of total.

In [ ]:
simplified = agb.sobol_simplify_model(
    total_inventory, # The model
    impacts, # Impacts to consider
    functional_unit=functional_value,
    
    n=10000, # For large model, you may test other value and ensure ST and sum(S1) are close to 1.0 
    fixed_mode = agb.FixedParamMode.MEDIAN, # We replace minor parameters by median by default,
    min_ratio=0.8, # Min ratio of variability to explain
    num_digits=3)

In [ ]:
# Let's look at the expression for first impact again 
# much simpler ! 
simplified[0].expr

Finally, we can compare the distribution of those simplified model against the full model. We provide a function for graphical display of it, and compuation of de R-Square score.


In [ ]:
agb.compare_simplified(
    total_inventory, 
    impacts, 
    simplified,
    functional_unit=functional_value)

### Mining techniques

In [ ]:
mining_shares = {'open_pit':.161/.935,
                'underground':.200/.935,
                'ISL':0.574/.935}

### Mining energy mix

In [ ]:
mining_energy_shares = {'diesel':0.3,
                        'electricity':0.7}

In [ ]:
# Total energy for extraction : where does that come from ???

a_op = 274
b_op = -0.482

a_ug = 317
b_ug = -0.176

a_is = 220
b_is = -0.0485

extraction_energy_open_pit = a_op * exp(b_op * ore_grade) / recovery_rate
extraction_energy_underground = a_ug * exp(b_ug * ore_grade) / recovery_rate
extraction_energy_ISL = a_is * exp(b_is * ore_grade) / recovery_rate

In [ ]:
mining_electricity_switch = newEnumParam(
    'mining_electricity_switch', 
    label='Mining electricity, grid or diesel',
    values=['dieselgenerator',
            'grid'], # You can provide a statistical weight for each value, by using a dict
    default='dieselgenerator', 
    description="Choice of electricity source for mining",
    dbname=USER_DB)

### Emissions

In [ ]:
# Emissions of radon-222 from tailings
tailings_Rn222 = newFloatParam(
    'tailings_Rn222', 
    default=0.01951, min=0.01, max=1,
    distrib=DistributionType.TRIANGLE, # Distribution type, linear by default
    description="Rn222 from tailings, in Bq/s",
    label="Rn222 from tailings, in Bq/s",
    unit='Bq/s',
    dbname=USER_DB)

## Concentration phase 

The following parameters are included:

- xxx


In [ ]:
# Conversion phase, heat consumption
conversion_heat = newFloatParam(
    'conversion_heat', 
    default=26, min=26, max=665,
    distrib=DistributionType.TRIANGLE,
    label='Conversion heat input',
    unit='kWh/kg U in UF6',
    description="Heat required to convert 1 kg of UF6",
    dbname=USER_DB)

## Refining stage 

### List of all parameters 

In [ ]:
list_parameters() # recap of all parameters declared in the model 

In [ ]:
# And because a figure is worth 1000 words
utils._plot_params([p for p in params._param_registry().all() if p.distrib in ['linear', 'triangle', 'lognormal']], columns=5, size=2000, figsize=(16,9))
plt.savefig('output/parameters.svg')

# Transforming and updating parametrized inventories

In [ ]:
# Mining and milling
tailings         = findActivity('Tailing, from uranium milling, WNA', loc='EUR', db_name='UNEP_IRP_EUR')
open_pit         = findActivity('Uranium mine operation, open cast, WNA', loc='GLO', db_name='UNEP_IRP_EUR')
underground      = findActivity('Uranium mine operation, underground, WNA', loc='GLO', db_name='UNEP_IRP_EUR')
ISL              = findActivity('Uranium mine operation, in-situ leaching, WNA', loc='GLO', db_name='UNEP_IRP_EUR')

In [ ]:
# Copy to preserve the non-parametrized inventories 
open_pit_p = copyActivity(
    USER_DB,
    open_pit,
    'Uranium mine operation, open cast, parameterized')

In [ ]:
# We replace fixed values by parameters in the LCI 
# Mining techniques, 3 activities
open_pit_p.updateExchanges({
    # This is electricity
    'market for diesel, burned in diesel-electric generating set, 10MW*': dict(amount=mining_energy_shares['electricity'] * extraction_energy_open_pit,
                                                                              input=mining_elec_mix),
    # This is diesel used as fuel
    'market for diesel, burned in building machine*': mining_energy_shares['diesel'] * extraction_energy_open_pit
}
)

In [ ]:
# Update inventories 